In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.data import load
import time
import pickle

from util import *

In [2]:
# data = load_complete('data/lyrics_380k.csv')
data = load_filtered('data/lyrics_380k.csv')

154781


## POS - tagging

In [7]:
tagdict = load('help/tagsets/upenn_tagset.pickle')
pos_list = list(tagdict.keys()) + ['#']
pos_list = {k: v for v, k in enumerate(pos_list)}

In [8]:
'''
POS-TAG Featurizer

Takes in a lyric, and performs POS tagging.
Returns the distribution of POS tags.
No pre-processing is performed on the lyric
'''
def pos_tag_feat(lyric, ind, mat):
#     result = np.zeros(len(pos_list))
    result = mat[ind]
    new_lyric = lyric 
    tok_lyric = nltk.word_tokenize(new_lyric)
    tag_lyric=  nltk.pos_tag(tok_lyric)
    for tag in tag_lyric:
        pos_tag = tag[1]
        try:
            ind = pos_list[pos_tag]
            result[ind] += 1
        except:
            print('Oops', tag)
    if np.sum(result) > 0.0:
        result = result / np.sum(result)
    return result

### Make POS feature matrix

In [9]:
NUM_LYRIC = 154781
NUM_FEAT = 46

pos_mat = np.zeros((NUM_LYRIC, NUM_FEAT))
num = 0
for artist in data:
    for i in range(len(data[artist]['lyrics'])):
        lyric = data[artist]['lyrics'][i]
        ind = data[artist]['indices'][i]
        pos_tag_feat(lyric, ind, pos_mat)
        num += 1
        if num % 500 == 0:
            print(num)
np.save('data/features_filtered/pos_feature.npy', pos_mat)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500
45000
45500
46000
46500
47000
47500
48000
48500
49000
49500
50000
50500
51000
51500
52000
52500
53000
53500
54000
54500
55000
55500
56000
56500
57000
57500
58000
58500
59000
59500
60000
60500
61000
61500
62000
62500
63000
63500
64000
64500
65000
65500
66000
66500
67000
67500
68000
68500
69000
69500
70000
70500
71000
71500
72000
72500
73000
73500
74000
74500
75000
75500
76000
76500
77000
77500
78000
78500
79000
79500
80000
80500
81000
81500
82000
82500
83000
83500
84000
84500
85000


In [54]:
# start = time.time()
# i = 0
# for artist in data:
#     for lyric in data[artist]['lyrics']:
#         pos_tag_feat(lyric)
#         i+= 1
#         if i == 1000:
#             break
#     if i == 1000:
#         break
# end = time.time()
# print(end - start)

29.417855978012085


## Length Features

In [3]:
'''
Punctuation mark counts
Lines per lyric
Words per lyric
avg Unique words per line
'''
def length_feat(lyric, ind, mat):
    count = lambda l1,l2: sum([1 for x in l1 if x in l2])
    table = str.maketrans({key: None for key in string.punctuation})
    t = lyric.translate(table) 
    t = t.lower()
    lines = t.splitlines()
    word_split = [l.split() for l in lines]
    
    punct_count = count(lyric,set(string.punctuation)) 
    line_count = len(lines)
    if line_count == 0:
        words_per_song = 0
        avg_unique_word_per_line = 0
    else:
        words_per_song = np.sum([len(l) for l in lines])
        avg_unique_word_per_line = np.mean([len(set(l)) for l in lines])
    mat[ind] =  [punct_count, line_count, words_per_song, avg_unique_word_per_line]


In [5]:
NUM_LYRIC = 154781
NUM_FEAT = 4

length_mat = np.zeros((NUM_LYRIC, NUM_FEAT))
num = 0
for artist in data:
    for i in range(len(data[artist]['lyrics'])):
        lyric = data[artist]['lyrics'][i]
        ind = data[artist]['indices'][i]
        length_feat(lyric, ind, length_mat)
        num += 1
        if num % 5000 == 0:
            print(num)

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000


In [6]:
np.save(file="data/features_filtered/length_feature.npy", arr = length_mat)

In [5]:
start = time.time()
i = 0
for artist in data:
    for lyric in data[artist]['lyrics']:
#         print(i)
        length_feat(lyric)
        i+= 1
        if i%10000 == 0:
            print(i)
end = time.time()
print(end - start)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
46.78280806541443


In [3]:
pickle.dump( data, open( "save.p", "wb" ), protocol=2 )

In [3]:
max_num = 0
for a in data:
    max_num = max(max_num, np.max(data[a]['indices']))
print(max_num)

In [31]:
from scipy.sparse import lil_matrix

In [32]:
x = lil_matrix((280000,6000))

np.save('temp', x)

In [33]:
x[1]  = np.ones(6000)

In [38]:
x
np.save('temp', x)